In [81]:
import os
from io import StringIO
import dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from langchain_openai import AzureChatOpenAI

dotenv.load_dotenv()

True

In [82]:
model: AzureChatOpenAI = None
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
Look at the image below and use it as input. Generate a response based on the image.
You are an expert in ontology engineering. Generate an OWL ontology based on the following domain description:
Define classes, data properties, and object properties.
Include domain and range for each property.
Provide the output in OWL (XML) format and only output the ontology and nothing else"""

task_prompt = """Generate an ontology based on the following domain description of test methodology for tensiles. This is the existing ontology that you need to extend if necessary:

"""

In [83]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
# model = AzureChatOpenAI(
#     azure_ad_token_provider=token_provider,
#     azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
#     azure_deployment=os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
#     openai_api_version=os.getenv("AZURE_OPENAI_VERSION"),
#     temperature=0,
#     openai_api_type="azure_ad"
# )

model = AzureChatOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_deployment=os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
        openai_api_version=os.getenv("AZURE_OPENAI_VERSION"),
        temperature=0
    )

In [84]:
import base64
from io import BytesIO
from PIL import Image

def convert_to_base64(pil_image):
    """
    Convert PIL images to Base64 encoded strings

    :param pil_image: PIL image
    :return: Base64 string
    """
    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")  # You can change the format if needed
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

def load_image(image_path):
    """
    Load an image from a file path

    :param image_path: Path to the image
    :return: PIL image
    """
    return Image.open(image_path)

In [85]:
image_path = "../../data/images/D412-16(2021)-14.jpg"

onthology_file_path="onthology.xml"

def process_file(image_file_path, onthology_file_path):
    image_data = convert_to_base64(load_image(image_file_path))
    with open(onthology_file_path, "r") as file:
        ontology_content = file.read()

    message = HumanMessage(
        content=[
            {"type": "text", "text": system_prompt + task_prompt + ontology_content},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
            },
        ],
    )

    response = model.invoke([message])
    print(response.content)

    with open(onthology_file_path, "w") as file:
        new_ontology = response.content.replace("```xml", "").replace("```", "")
        file.write(new_ontology)


process_file(image_path, onthology_file_path)

# folder_path = "../../data/images"

# for file in os.listdir(folder_path):
#     if file.endswith(".jpg"):
#         process_file(os.path.join(folder_path, file), onthology_file_path)


```xml
<rdf:RDF xmlns="http://www.example.org/tensile#"
     xml:base="http://www.example.org/tensile"
     xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
     xmlns:owl="http://www.w3.org/2002/07/owl#"
     xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
     xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
    <owl:Ontology rdf:about="http://www.example.org/tensile"/>

    <!-- Classes -->
    <owl:Class rdf:about="http://www.example.org/tensile#TestSpecimen"/>
    <owl:Class rdf:about="http://www.example.org/tensile#RingSpecimen"/>
    <owl:Class rdf:about="http://www.example.org/tensile#TensileTest"/>
    <owl:Class rdf:about="http://www.example.org/tensile#Measurement"/>
    <owl:Class rdf:about="http://www.example.org/tensile#Procedure"/>
    <owl:Class rdf:about="http://www.example.org/tensile#Calculation"/>
    <owl:Class rdf:about="http://www.example.org/tensile#VulcanizedRubber"/>
    <owl:Class rdf:about="http://www.example.org/tensile#ThermoplasticElastomer"/>
   